# Testing: quick_find_parameters()

In [2]:
from simulation.mean_var_autocorr import check_biological_appropriateness, find_parameters
import numpy as np
import tqdm
%load_ext autoreload
%autoreload 2

# Define a function to try different values
def find_biologically_appropriate_parameters(target_cv_initial=1.7, mu_target_initial=5, max_attempts=10):
    # Try different combinations of CV and mean until we find biologically appropriate values
    for attempt in range(max_attempts):
        # On first attempt, use the initial values
        if attempt == 0:
            target_cv = target_cv_initial
            mu = mu_target_initial
        # On subsequent attempts, adjust the values
        else:
            # Try reducing CV or increasing mean to improve Fano factor and CV
            target_cv = target_cv_initial * (0.9 - attempt * 0.05)  # Gradually reduce CV
            mu = mu_target_initial * (1.1 + attempt * 0.1)         # Gradually increase mean
        
        # Calculate corresponding variance
        variance = (target_cv * mu)**2
        
        print(f"\nAttempt {attempt+1}/{max_attempts}")
        print(f"Testing: CV = {target_cv:.2f}, Mean = {mu:.2f}, Variance = {variance:.2f}")
        
        # Check if these values are biologically appropriate
        is_appropriate = check_biological_appropriateness(variance, mu)
        
        if is_appropriate:
            print(f"✅ Found biologically appropriate values: CV = {target_cv:.2f}, Mean = {mu:.2f}, Variance = {variance:.2f}")
            return target_cv, mu, variance
    
    # If we reach here, we couldn't find appropriate parameters
    print("❌ Could not find biologically appropriate parameters")
    return None, None, None

# Function to pre-check all CV ratios for biological appropriateness
def find_parameters_without_skipping(initial_cv_normal=1.7, initial_mu_target=5, cv_ratios=np.arange(0.1, 3.0, 0.01), max_attempts=5):
    for attempt in range(max_attempts):
        # On first attempt, use the initial values
        if attempt == 0:
            cv_normal = initial_cv_normal
            mu = initial_mu_target
        # On subsequent attempts, adjust the values
        else:
            # Try decreasing CV and increasing mean to improve all conditions
            cv_normal = initial_cv_normal * (0.85 - attempt * 0.05)  # More aggressive CV reduction
            mu = initial_mu_target * (1.3 + attempt * 0.2)          # More aggressive mean increase
        
        # Calculate normal variance
        variance_normal = (cv_normal * mu)**2
        
        print(f"\nTesting parameters: CV normal = {cv_normal:.2f}, Mean = {mu:.2f}")
        
        # Check if normal condition is biologically appropriate
        is_normal_appropriate = check_biological_appropriateness(variance_normal, mu)
        if not is_normal_appropriate:
            print(f"⚠️ Normal condition not biologically appropriate. Trying different parameters...")
            continue
        
        # Check all CV ratios for the stress condition
        all_appropriate = True
        inappropriate_ratios = []
        
        for ratio in tqdm.tqdm(cv_ratios, desc="Checking all CV ratios"):
            cv_stress = ratio * cv_normal
            variance_stress = (cv_stress * mu)**2
            
            is_stress_appropriate = check_biological_appropriateness(variance_stress, mu)
            if not is_stress_appropriate:
                all_appropriate = False
                inappropriate_ratios.append(ratio)
        
        if all_appropriate:
            print(f"✅ Found parameters where all CV ratios are biologically appropriate!")
            print(f"CV normal = {cv_normal:.2f}, Mean = {mu:.2f}, Variance normal = {variance_normal:.2f}")
            return cv_normal, mu, variance_normal
        else:
            print(f"⚠️ {len(inappropriate_ratios)} out of {len(cv_ratios)} CV ratios are not biologically appropriate")
            print(f"First few inappropriate ratios: {inappropriate_ratios[:5]}")
    
    print("❌ Could not find parameters where all CV ratios are biologically appropriate")
    print("You may need to adjust the range of CV ratios or try more extreme parameter values")
    return None, None, None


## Let's get some biologically sensible parameters firt

In [3]:
cv_ratio = np.arange(0.5, 2.0, 0.01)  

# Set initial parameters for search
initial_cv_normal = 0.8  # Try a lower initial CV for normal condition
initial_mu_target = 15   # Try a higher initial mean
autocorr_target = 0.5    # Autocorrelation (same for both conditions)

# Try to find parameters that work for all CV ratios
print("Finding parameters where all CV ratios are biologically appropriate...")
target_cv_normal, mu_target, variance_normal = find_parameters_without_skipping(
    initial_cv_normal=initial_cv_normal, 
    initial_mu_target=initial_mu_target,
    cv_ratios=cv_ratio,
    max_attempts=10  # Increase max attempts to try more parameter combinations
)

# If we couldn't find parameters that work for all ratios, use the original function
if target_cv_normal is None:
    print("\nFalling back to finding biologically appropriate parameters for normal condition only...")
    target_cv_normal, mu_target, variance_normal = find_biologically_appropriate_parameters(
        target_cv_initial=initial_cv_normal, 
        mu_target_initial=initial_mu_target
    )

# Initial parameters for telegraph model
parameters = {
    "stress": {"sigma_b": 46.7},
    "normal": {"sigma_b": 5.4}
}


Finding parameters where all CV ratios are biologically appropriate...

Testing parameters: CV normal = 0.80, Mean = 15.00
✅ System is biologically appropriate with Fano factor: 9.60, CV: 0.80


Checking all CV ratios: 100%|██████████| 150/150 [00:00<00:00, 95658.45it/s]

✅ System is biologically appropriate with Fano factor: 2.40, CV: 0.40
✅ System is biologically appropriate with Fano factor: 2.50, CV: 0.41
✅ System is biologically appropriate with Fano factor: 2.60, CV: 0.42
✅ System is biologically appropriate with Fano factor: 2.70, CV: 0.42
✅ System is biologically appropriate with Fano factor: 2.80, CV: 0.43
✅ System is biologically appropriate with Fano factor: 2.90, CV: 0.44
✅ System is biologically appropriate with Fano factor: 3.01, CV: 0.45
✅ System is biologically appropriate with Fano factor: 3.12, CV: 0.46
✅ System is biologically appropriate with Fano factor: 3.23, CV: 0.46
✅ System is biologically appropriate with Fano factor: 3.34, CV: 0.47
✅ System is biologically appropriate with Fano factor: 3.46, CV: 0.48
✅ System is biologically appropriate with Fano factor: 3.57, CV: 0.49
✅ System is biologically appropriate with Fano factor: 3.69, CV: 0.50
✅ System is biologically appropriate with Fano factor: 3.81, CV: 0.50
✅ System is biologic

## Test out the function by trying to find parameters across the CV range

In [4]:
# Test the quick_find_parameters function across CV ratios
import time
import numpy as np
from simulation.mean_cv_autocorr import quick_find_parameters
from simulation.mean_var_autocorr import check_biological_appropriateness

%load_ext autoreload
%autoreload 2

# Define test parameters
sigma_b_test = 1.0
mu_target_test = 12
autocorr_target_test = 0.5
base_cv = 0.8


# Define a range of CV values to test
cv_ratios = np.arange(0.5, 2.0, 0.01)

results = []

for ratio in tqdm.tqdm(cv_ratios, desc="Testing CV ratios"):
    cv_target_test = ratio * base_cv
    variance_test = (cv_target_test * mu_target_test)**2
    
    print(f"\nTesting CV ratio: {ratio:.2f}, CV: {cv_target_test:.2f}")
    print(f"Mean: {mu_target_test:.2f}, Variance: {variance_test:.2f}")
    
    # Check if biologically appropriate
    is_appropriate = check_biological_appropriateness(variance_test, mu_target_test)
    if not is_appropriate:
        print(f"⚠️ Parameters not biologically appropriate for ratio {ratio:.2f}. Skipping.")
        continue

    # Test quick_find_parameters
    start_time = time.time()
    try:
        quick_result = quick_find_parameters(
            sigma_b=sigma_b_test,
            mu_target=mu_target_test,
            autocorr_target=autocorr_target_test,
            cv_target=cv_target_test
        )
        quick_time = time.time() - start_time
        print(f"quick_find_parameters: rho={quick_result[0]:.4f}, sigma_u={quick_result[1]:.4f}, d={quick_result[2]:.4f}, time={quick_time:.4f}s")
    except Exception as e:
        print(f"Error with quick_find_parameters: {str(e)}")
        quick_time = None
        quick_result = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Testing CV ratios:   0%|          | 0/150 [00:00<?, ?it/s]/home/ianyang/stochastic_simulations/src/simulation/mean_cv_autocorr.py:37: RuntimeWarning: overflow encountered in exp
  return -1/np.exp(1) + (-(d_val*mu_target*(1 + np.exp((autocorr_target*d_val*(d_val*mu_target - (-2 + mu_target + 2*cv_sq_target*mu_target)*sigma_b))/(d_val*mu_target + sigma_b - cv_sq_target*mu_target*sigma_b))*(-1 + cv_sq_target*mu_target))) + (-1 + cv_sq_target*mu_target**2 + cv_sq_target**2*mu_target**2 + np.exp((autocorr_target*d_val*(d_val*mu_target - (-2 + mu_target + 2*cv_sq_target*mu_target)*sigma_b))/(d_val*mu_target + sigma_b - cv_sq_target*mu_target*sigma_b))*(-1 + cv_sq_target*mu_target)**2)*sigma_b)/(cv_sq_target*np.exp(autocorr_target*d_val)*mu_target*(-(d_val*mu_target) + (-2 + mu_target + 2*cv_sq_target*mu_target)*sigma_b))
/home/ianyang/stochastic_simulations/src/simulation/mean_cv_autocorr.py:37: RuntimeWarning: invalid value encountered in add
  return -1/np.exp(1) + (-(d_val*mu_target*(1 +


Testing CV ratio: 0.50, CV: 0.40
Mean: 12.00, Variance: 23.04
✅ System is biologically appropriate with Fano factor: 1.92, CV: 0.40
Found valid solution for D=0.6931471805599453: rho=74.1677875689878, sigma_u=0.09144327778386657, d=5.662821962339524
Final d_value: 5.662821962339524, rho: 74.1677875689878, sigma_u: 0.09144327778386657
Found solution: rho=74.1678, sigma_u=0.0914, d=5.6628
quick_find_parameters: rho=74.1678, sigma_u=0.0914, d=5.6628, time=0.0278s

Testing CV ratio: 0.51, CV: 0.41
Mean: 12.00, Variance: 23.97
✅ System is biologically appropriate with Fano factor: 2.00, CV: 0.41
Found valid solution for D=0.6931471805599453: rho=81.89651484644351, sigma_u=0.09781087247307282, d=6.216653288523844
Final d_value: 6.216653288523844, rho: 81.89651484644351, sigma_u: 0.09781087247307282
Found solution: rho=81.8965, sigma_u=0.0978, d=6.2167
quick_find_parameters: rho=81.8965, sigma_u=0.0978, d=6.2167, time=0.0051s

Testing CV ratio: 0.52, CV: 0.42
Mean: 12.00, Variance: 24.92
✅ S

Testing CV ratios: 100%|██████████| 150/150 [00:00<00:00, 314.36it/s]

Found valid solution for D=0.6931471805599453: rho=325.51590016320887, sigma_u=1.2679542305520446, d=11.960702137713731
Final d_value: 11.960702137713731, rho: 325.51590016320887, sigma_u: 1.2679542305520446
Found solution: rho=325.5159, sigma_u=1.2680, d=11.9607
quick_find_parameters: rho=325.5159, sigma_u=1.2680, d=11.9607, time=0.0049s

Testing CV ratio: 1.35, CV: 1.08
Mean: 12.00, Variance: 167.96
✅ System is biologically appropriate with Fano factor: 14.00, CV: 1.08
Found valid solution for D=0.6931471805599453: rho=313.0684891328285, sigma_u=1.3033094315212956, d=11.326763310233
Final d_value: 11.326763310233, rho: 313.0684891328285, sigma_u: 1.3033094315212956
Found solution: rho=313.0685, sigma_u=1.3033, d=11.3268
quick_find_parameters: rho=313.0685, sigma_u=1.3033, d=11.3268, time=0.0046s

Testing CV ratio: 1.36, CV: 1.09
Mean: 12.00, Variance: 170.46
✅ System is biologically appropriate with Fano factor: 14.20, CV: 1.09
Found valid solution for D=0.6931471805599453: rho=302.0

``quick_find_parameter()`` seems to be working fine with biologically relevant systems

## Let's try and run this for 2 conditions

In [14]:
import tqdm
from simulation.mean_var_autocorr import check_biological_appropriateness 
from simulation.mean_cv_autocorr import quick_find_parameters
###############################################################################
# Generate Synthetic Data
# 1) Define target mean, CV and autocorrelations, and some parameters to start with
###############################################################################

# Set initial parameters based on our testing in the notebook
# Initial parameters

parameters = {
    "stress": {"sigma_b": 46.7},
    "normal": {"sigma_b": 5.4}
}
target_cv_normal = 0.5  # initial CV for normal condition
mu_target = 12   # Try a higher initial mean
autocorr_target = 0.5    # Autocorrelation time (same for both conditions)

# Try to find parameters that work for all CV ratios
print("Finding parameters where all CV ratios are biologically appropriate...")
# Define a narrower CV ratio range based on testing
cv_ratio = np.arange(0.5, 2.0, 0.01)  # Narrower range that's more likely to be biologically appropriate

print(f"\nStarting simulations with:")
print(f"Normal CV: {target_cv_normal:.2f}, Mean: {mu_target:.2f}, Autocorrelation Time: {autocorr_target:.2f}")

###############################################################################
# 2) Loop over different CV ratios
###############################################################################

# Track inappropriate ratios for reporting
inappropriate_ratios = []
param_finding_failures = []  # Track any failures in parameter finding

for ratio in tqdm.tqdm(cv_ratio, desc="Running CV Ratio Simulations"):
    # For the stress condition, we define CV by ratio
    target_cv_stress = ratio * target_cv_normal
    
    # Calculate corresponding variance values for reporting only
    variance_stress = (target_cv_stress * mu_target)**2

    
    print(f"\nTesting CV ratio: {ratio:.2f}, Stress CV: {target_cv_stress:.2f}, Normal CV: {target_cv_normal:.2f}")
    print(f"Corresponding variances - Stress: {variance_stress:.2f}, Normal: {variance_normal:.2f}")
    
    # Double-check if stress parameters are biologically appropriate
    is_stress_appropriate = check_biological_appropriateness(variance_stress, mu_target)

    # If stress parameters are not biologically appropriate, track it and skip
    if not is_stress_appropriate:
        print(f"⚠️ Stress parameters are not biologically appropriate for ratio {ratio:.2f}. Skipping.")
        inappropriate_ratios.append(ratio)
        continue
    
    # Store results for each condition
    results = {}
    success = True  # Track if both conditions succeed
    
    
    for condition, param in parameters.items():
        # Decide which CV to use for this condition
        if condition == "normal":
            cv_for_condition = target_cv_normal
        else:  # condition == "stress"
            cv_for_condition = target_cv_stress

        try:
            # Fix mean, CV, and autocorrelation
            # Pass the sigma_b value from the param dictionary, not the whole dictionary
            rho, sigma_u, d = quick_find_parameters(
                param.get('sigma_b'), mu_target=mu_target, autocorr_target=autocorr_target, 
                cv_target=cv_for_condition
            )

            results[condition] = {"rho": rho, "sigma_u": sigma_u, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")
            param_finding_failures.append({"condition": condition, "sigma_b": param.get('sigma_b'), 'mu_target': mu_target, 'autocorr_target': autocorr_target, 'cv_for_condition': cv_for_condition, "error": str(e)})
            success = False
    
    # If any condition failed, skip to the next ratio
    if not success:
        print(f"Skipping ratio {ratio:.2f} due to parameter finding failure")
        continue


    # update parameter sets 
    parameter_sets = [
        {"sigma_u": results["stress"]["sigma_u"], 
        "sigma_b": parameters["stress"]['sigma_b'], 
        "rho": results["stress"]['rho'], 
        "d": results["stress"]['d'], 
        "label": 0},
        
        {"sigma_u": results["normal"]["sigma_u"], 
        "sigma_b": parameters["normal"]['sigma_b'], 
        "rho": results["normal"]['rho'], 
        "d": results["normal"]['d'], 
        "label": 1}
    ]
    

    # Output the results
    print("Updated Parameter Sets:", parameter_sets)

Finding parameters where all CV ratios are biologically appropriate...

Starting simulations with:
Normal CV: 0.50, Mean: 12.00, Autocorrelation Time: 0.50


Running CV Ratio Simulations:   0%|          | 0/150 [00:00<?, ?it/s]

Running CV Ratio Simulations:  28%|██▊       | 42/150 [00:00<00:00, 173.80it/s]


Testing CV ratio: 0.50, Stress CV: 0.25, Normal CV: 0.50
Corresponding variances - Stress: 9.00, Normal: 144.00
⚠️ WARNING: Fano factor 0.75 is outside the acceptable range (1, 20).
Final d_value: 1.953883925274376, rho: 15.326473982033916, sigma_u: -16.17335144023997
Found solution: rho=15.3265, sigma_u=-16.1734, d=1.9539
[STRESS] ✅ Found: {'rho': 15.326473982033916, 'sigma_u': -16.17335144023997, 'd': 1.953883925274376}
Found valid solution for D=0.6931471805599453: rho=53.40433041385754, sigma_u=4.784417815256124, d=2.359678198810377
Final d_value: 2.359678198810377, rho: 53.40433041385754, sigma_u: 4.784417815256124
Found solution: rho=53.4043, sigma_u=4.7844, d=2.3597
[NORMAL] ✅ Found: {'rho': 53.40433041385754, 'sigma_u': 4.784417815256124, 'd': 2.359678198810377}
Updated Parameter Sets: [{'sigma_u': -16.17335144023997, 'sigma_b': 46.7, 'rho': 15.326473982033916, 'd': 1.953883925274376, 'label': 0}, {'sigma_u': 4.784417815256124, 'sigma_b': 5.4, 'rho': 53.40433041385754, 'd': 2.

Running CV Ratio Simulations:  47%|████▋     | 70/150 [00:00<00:00, 214.86it/s]


Testing CV ratio: 0.92, Stress CV: 0.46, Normal CV: 0.50
Corresponding variances - Stress: 30.47, Normal: 144.00
✅ System is biologically appropriate with Fano factor: 2.54, CV: 0.46
Final d_value: None, rho: None, sigma_u: None
Could not find a valid solution for parameter d
[STRESS] ❌ No suitable solution found.
Found valid solution for D=0.6931471805599453: rho=53.40433041385754, sigma_u=4.784417815256124, d=2.359678198810377
Final d_value: 2.359678198810377, rho: 53.40433041385754, sigma_u: 4.784417815256124
Found solution: rho=53.4043, sigma_u=4.7844, d=2.3597
[NORMAL] ✅ Found: {'rho': 53.40433041385754, 'sigma_u': 4.784417815256124, 'd': 2.359678198810377}
Skipping ratio 0.92 due to parameter finding failure

Testing CV ratio: 0.93, Stress CV: 0.47, Normal CV: 0.50
Corresponding variances - Stress: 31.14, Normal: 144.00
✅ System is biologically appropriate with Fano factor: 2.59, CV: 0.47
Final d_value: None, rho: None, sigma_u: None
Could not find a valid solution for parameter

Running CV Ratio Simulations:  87%|████████▋ | 131/150 [00:00<00:00, 264.08it/s]


Testing CV ratio: 1.20, Stress CV: 0.60, Normal CV: 0.50
Corresponding variances - Stress: 51.84, Normal: 144.00
✅ System is biologically appropriate with Fano factor: 4.32, CV: 0.60
Final d_value: None, rho: None, sigma_u: None
Could not find a valid solution for parameter d
[STRESS] ❌ No suitable solution found.
Found valid solution for D=0.6931471805599453: rho=53.40433041385754, sigma_u=4.784417815256124, d=2.359678198810377
Final d_value: 2.359678198810377, rho: 53.40433041385754, sigma_u: 4.784417815256124
Found solution: rho=53.4043, sigma_u=4.7844, d=2.3597
[NORMAL] ✅ Found: {'rho': 53.40433041385754, 'sigma_u': 4.784417815256124, 'd': 2.359678198810377}
Skipping ratio 1.20 due to parameter finding failure

Testing CV ratio: 1.21, Stress CV: 0.61, Normal CV: 0.50
Corresponding variances - Stress: 52.71, Normal: 144.00
✅ System is biologically appropriate with Fano factor: 4.39, CV: 0.61
Final d_value: None, rho: None, sigma_u: None
Could not find a valid solution for parameter

Running CV Ratio Simulations: 100%|██████████| 150/150 [00:00<00:00, 247.01it/s]


Testing CV ratio: 1.81, Stress CV: 0.91, Normal CV: 0.50
Corresponding variances - Stress: 117.94, Normal: 144.00
✅ System is biologically appropriate with Fano factor: 9.83, CV: 0.91
Final d_value: None, rho: None, sigma_u: None
Could not find a valid solution for parameter d
[STRESS] ❌ No suitable solution found.
Found valid solution for D=0.6931471805599453: rho=53.40433041385754, sigma_u=4.784417815256124, d=2.359678198810377
Final d_value: 2.359678198810377, rho: 53.40433041385754, sigma_u: 4.784417815256124
Found solution: rho=53.4043, sigma_u=4.7844, d=2.3597
[NORMAL] ✅ Found: {'rho': 53.40433041385754, 'sigma_u': 4.784417815256124, 'd': 2.359678198810377}
Skipping ratio 1.81 due to parameter finding failure

Testing CV ratio: 1.82, Stress CV: 0.91, Normal CV: 0.50
Corresponding variances - Stress: 119.25, Normal: 144.00
✅ System is biologically appropriate with Fano factor: 9.94, CV: 0.91
Final d_value: None, rho: None, sigma_u: None
Could not find a valid solution for paramet

In [28]:
# these are parameters that failed to find a solution, let's try to modify the quick_find_parameters function to resolve issues that arised
results = {}
success = True  # Track if all conditions succeed
for param in param_finding_failures:
    try:
        condition = param['condition']
        # Fix mean, CV, and autocorrelation
        # Pass the sigma_b value from the param dictionary, not the whole dictionary
        rho, sigma_u, d = quick_find_parameters(
            param.get('sigma_b'), mu_target=param.get('mu_target'), autocorr_target=param.get('autocorr_target'), cv_target=param.get('cv_for_condition')
        )

        results[condition] = {"rho": rho, "sigma_u": sigma_u, "d": d}
        print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
    except ValueError as e:
        print(f'{e}')
        print(f"[{condition.upper()}] ❌ No suitable solution found.")
        # param_finding_failures.append({"condition": condition, "parameters": param, 'mu_target': mu_target, 'autocorr_target': autocorr_target, 'cv_for_condition': cv_for_condition, "error": str(e)})
        success = False

    # If any condition failed, skip to the next ratio
    if not success:
        print(f"Skipping CV condition {param['cv_for_condition']:.2f} due to parameter finding failure")
        continue


# # update parameter sets 
# parameter_sets = [
#     {"sigma_u": results["stress"]["sigma_u"], 
#     "sigma_b": parameters["stress"]['sigma_b'], 
#     "rho": results["stress"]['rho'], 
#     "d": results["stress"]['d'], 
#     "label": 0},
    
#     {"sigma_u": results["normal"]["sigma_u"], 
#     "sigma_b": parameters["normal"]['sigma_b'], 
#     "rho": results["normal"]['rho'], 
#     "d": results["normal"]['d'], 
#     "label": 1}
# ]


# Output the results
# print("Updated Parameter Sets:", parameter_sets)

Final d_value: None, rho: None, sigma_u: None
Could not find a valid solution for parameter d
[STRESS] ❌ No suitable solution found.
Skipping CV condition 0.34 due to parameter finding failure
Final d_value: 2.160660417068302, rho: 8274359.062606401, sigma_u: 14903288.917944375
[STRESS] ✅ Found: {'rho': 8274359.062606401, 'sigma_u': 14903288.917944375, 'd': 2.160660417068302}
Skipping CV condition 0.36 due to parameter finding failure
Final d_value: 2.3299416039249676, rho: 88754732.47955015, sigma_u: 148245657.53469977
[STRESS] ✅ Found: {'rho': 88754732.47955015, 'sigma_u': 148245657.53469977, 'd': 2.3299416039249676}
Skipping CV condition 0.37 due to parameter finding failure
Final d_value: 2.501564264912466, rho: 84927207.93348245, sigma_u: 132120638.28242943
[STRESS] ✅ Found: {'rho': 84927207.93348245, 'sigma_u': 132120638.28242943, 'd': 2.501564264912466}
Skipping CV condition 0.37 due to parameter finding failure
Final d_value: 2.6755244470214143, rho: 25132936.828128636, sigma_u

/home/ianyang/stochastic_simulations/src/simulation/mean_cv_autocorr.py:52: RuntimeWarning: overflow encountered in exp
  if result[2] == 1:  # Check if converged
/home/ianyang/stochastic_simulations/src/simulation/mean_cv_autocorr.py:52: RuntimeWarning: invalid value encountered in scalar add
  if result[2] == 1:  # Check if converged
